In [2]:
!git clone https://github.com/SAI990323/BIGRec.git


Cloning into 'BIGRec'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 116 (delta 57), reused 77 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (116/116), 1.97 MiB | 16.93 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [3]:
%cd BIGRec

/content/BIGRec


In [5]:
!pip install -r requirements.txt


  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-6s_sl1yy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-6s_sl1yy
  Resolved https://github.com/huggingface/peft.git to commit 2f063e6342235842a00bca150a98854b89117986
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Cannot install peft 0.14.1.dev0 (from git+https://github.com/huggingface/peft.git) and peft==0.3.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested peft==0.3.0
    The user requested peft 0.14.1.dev0 (from git+https://github.com/huggingface/peft.git)

To fix this you could try to:
1. lo

In [8]:
import os
os.getcwd()

'/content/BIGRec'

In [9]:
%cd data/movie/

/content/BIGRec/data/movie


In [12]:
f = open('ratings.dat', 'r')
data = f.readlines()
f = open('movies.dat', 'r', encoding='ISO-8859-1')
movies = f.readlines()
movie_names = [_.split('::')[1] for _ in movies]
movie_ids = [_.split('::')[0] for _ in movies]
movie_dict = dict(zip(movie_ids, movie_names))
id_mapping = dict(zip(movie_ids, range(len(movie_ids))))

In [13]:
interaction_dicts = dict()
for line in data:
    user_id, movie_id, rating, timestamp = line.split('::')
    if user_id not in interaction_dicts:
        interaction_dicts[user_id] = {
            'movie_id': [],
            'rating': [],
            'timestamp': [],
            'movie_title': [],
        }
    interaction_dicts[user_id]['movie_id'].append(movie_id)
    interaction_dicts[user_id]['rating'].append(int(float(rating) > 3.0))
    interaction_dicts[user_id]['timestamp'].append(timestamp)
    interaction_dicts[user_id]['movie_title'].append(movie_dict[movie_id])


In [14]:
with open('all.csv', 'w') as f:
    import csv
    writer = csv.writer(f)
    writer.writerow(['user_id', 'item_id', 'rating', 'timestamp', 'item_title'])
    for user_id, user_dict in interaction_dicts.items():
        writer.writerow([user_id, user_dict['movie_id'], user_dict['rating'], user_dict['timestamp'], user_dict['movie_title']])


In [15]:
sequential_interaction_list = []
seq_len = 10
for user_id in interaction_dicts:
    temp = zip(interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'], interaction_dicts[user_id]['movie_title'])
    temp = sorted(temp, key=lambda x: int(x[2]))
    result = zip(*temp)
    interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'], interaction_dicts[user_id]['movie_title'] = [list(_) for _ in result]
    for i in range(10, len(interaction_dicts[user_id]['movie_id'])):
        sequential_interaction_list.append(
            [user_id, interaction_dicts[user_id]['movie_title'][i - seq_len: i],interaction_dicts[user_id]['movie_id'][i-seq_len:i], interaction_dicts[user_id]['rating'][i-seq_len:i], interaction_dicts[user_id]['movie_id'][i], interaction_dicts[user_id]['rating'][i], interaction_dicts[user_id]['timestamp'][i].strip('\n')]
        )
print(len(sequential_interaction_list))

683268


In [16]:
import csv
sequential_interaction_list = sorted(sequential_interaction_list, key=lambda x: int(x[-1]))
with open('./train.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_title', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[:int(len(sequential_interaction_list)*0.8)])
with open('./valid.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_title', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[int(len(sequential_interaction_list)*0.8):int(len(sequential_interaction_list)*0.9)])
with open('./test.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_title', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[int(len(sequential_interaction_list)*0.9):])

In [17]:
import json
import pandas as pd
import random
import numpy as np
def csv_to_json(input_path, output_path, sample=False):
    data = pd.read_csv(input_path)
    if sample:
        data = data.sample(n=5000, random_state=42).reset_index(drop=True)
        data.to_csv(output_path[:-5] + ".csv", index=False)
    json_list = []
    for index, row in data.iterrows():
        row['history_movie_id'] = eval(row['history_movie_id'])
        row['history_movie_title'] = eval(row['history_movie_title'])
        L = len(row['history_movie_id'])
        history = "The user has watched the following movies before:"
        for i in range(L):
            if i == 0:
                history += "\"" + row['history_movie_title'][i] + "\""
            else:
                history += ", \"" + row['history_movie_title'][i] + "\""
        target_movie_name = "\"" + movie_dict[str(row['movie_id'])] + "\""
        json_list.append({
            "instruction": "Given a list of movies the user has watched before, please recommend a new movie that the user likes to the user.",
            "input": f"{history}\n ",
            "output": target_movie_name,
        })

    with open(output_path, 'w') as f:
        json.dump(json_list, f, indent=4)


In [18]:
csv_to_json('./train.csv', './train.json')
csv_to_json('./valid.csv', './valid.json')
csv_to_json('./test.csv', './test.json')
csv_to_json('./valid.csv', './valid_5000.json', sample=True)
csv_to_json('./test.csv', './test_5000.json', sample=True)

In [20]:
%cd ..
os.getcwd()

/content/BIGRec


'/content/BIGRec'

In [21]:

# 设置超参数
seeds = [0, 1, 2]
learning_rates = [1e-4]
dropouts = [0.05]
samples = [1024]

# 循环运行模型
for seed in seeds:
    for lr in learning_rates:
        for dropout in dropouts:
            for sample in samples:
                print(f"lr: {lr}, dropout: {dropout}, seed: {seed},")
                # 构建命令
                command = f"CUDA_VISIBLE_DEVICES=$1 python train.py \
                    --base_model YOUR_LLAMA_PATH/ \
                    --train_data_path '[\"./data/movie/train.json\"]'   \
                    --val_data_path '[\"./data/movie/valid_5000.json\"]' \
                    --output_dir /model/movie/{seed}_{sample} \
                    --batch_size 128 \
                    --micro_batch_size 4 \
                    --num_epochs 50 \
                    --learning_rate {lr} \
                    --cutoff_len 512 \
                    --lora_r 8 \
                    --lora_alpha 16\
                    --lora_dropout {dropout} \
                    --lora_target_modules '[q_proj,v_proj]' \
                    --train_on_inputs \
                    --group_by_length \
                    --resume_from_checkpoint 'XXX' \
                    --seed {seed} \
                    --sample {sample}"
                # 执行命令
                os.system(command)


lr: 0.0001, dropout: 0.05, seed: 0,
lr: 0.0001, dropout: 0.05, seed: 1,
lr: 0.0001, dropout: 0.05, seed: 2,


In [24]:
!pip install fire

  Using cached fire-0.7.0.tar.gz (87 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=3bca929025ca1cbb91b64fd604d8aaba0c9bd982f6f5d1e942cd8abb55b4a45b
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [25]:
import subprocess

def run_command(command):
    result = subprocess.run(command, shell=True, text=True, capture_output=True)
    print("Status:", result.returncode)  # 打印返回状态码
    print("Output:", result.stdout)      # 打印标准输出
    if result.stderr:                    # 如果有标准错误输出，也打印它
        print("Errors:", result.stderr)

# 示例命令
command = f"CUDA_VISIBLE_DEVICES=$1 python train.py \
    --base_model YOUR_LLAMA_PATH/ \
    --train_data_path '[\"./data/movie/train.json\"]'   \
    --val_data_path '[\"./data/movie/valid_5000.json\"]' \
    --output_dir /model/movie/{seed}_{sample} \
    --batch_size 128 \
    --micro_batch_size 4 \
    --num_epochs 50 \
    --learning_rate {lr} \
    --cutoff_len 512 \
    --lora_r 8 \
    --lora_alpha 16\
    --lora_dropout {dropout} \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length \
    --resume_from_checkpoint 'XXX' \
    --seed {seed} \
    --sample {sample}"
run_command(command)

Status: 1
Output: 
Errors: Traceback (most recent call last):
  File "/content/BIGRec/train.py", line 10, in <module>
    from datasets import load_dataset, concatenate_datasets
ModuleNotFoundError: No module named 'datasets'

